In [1]:
import re
import numpy as np
import itertools
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.utils.data import random_split, RandomSampler
import random
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, mean_squared_error,  mean_absolute_error

In [2]:

#CALCULATING DELTA
df1 = pd.read_csv('cluster_contributions_ion.txt')
df2 = pd.read_csv('cluster_contributions_water.txt')

merged_df = pd.merge(df1, df2, on='Cluster', suffixes=('_file1', '_file2'))

merged_df['elec'] = merged_df['cls_elec_file1'] + (merged_df['cls_elec_file2'])
merged_df['ct'] = merged_df['ct_file1'] + (merged_df['ct_file2'])
merged_df['pauli'] = merged_df['mod_pauli_file1'] + (merged_df['mod_pauli_file2'])
merged_df['pol'] = merged_df['pol_file1'] + (merged_df['pol_file2'])
merged_df['disp'] = merged_df['disp_file1'] + (merged_df['disp_file2'])

result_df = merged_df[['ct', 'disp', 'elec', 'pol', 'pauli']]


result_df.to_csv('result_df.csv', index=False)

In [3]:
#EDA VS CMM

import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score

df = pd.read_csv('ion_water_mbe_eda_wb97xv_qzvppd.csv')

# Specify the ion-labels
br_df = df[df['ion_labels'] == 'F']
eda = br_df[['ct', 'disp', 'elec', 'pol', 'pauli', 'index']]  # Including the 'index' column


df_eda = pd.read_csv('ion_water_mbe_eda_cmm.csv')

if 'index' not in df_eda.columns:
    df_eda['index'] = range(1, len(df_eda) + 1)  

cmm = df_eda[df_eda['index'].isin(eda['index'])]

cmm_filtered = cmm[['ct', 'disp', 'elec', 'pol', 'pauli']]
eda_filtered = eda[['ct', 'disp', 'elec', 'pol', 'pauli']]

cmm_filtered.to_csv('cmm_filtered.csv', index=False)
eda_filtered.to_csv('eda_filtered.csv', index=False)

df1 = pd.read_csv('cluster_contributions_ion.txt')
df2 = pd.read_csv('cluster_contributions_water.txt')

merged_df = pd.merge(df1, df2, on='Cluster', suffixes=('_file1', '_file2'))

merged_df['elec'] = merged_df['cls_elec_file1'] + merged_df['cls_elec_file2']
merged_df['ct'] = merged_df['ct_file1'] + merged_df['ct_file2']
merged_df['pauli'] = merged_df['mod_pauli_file1'] + merged_df['mod_pauli_file2']
merged_df['pol'] = merged_df['pol_file1'] + merged_df['pol_file2']
merged_df['disp'] = merged_df['disp_file1'] + merged_df['disp_file2']

result_df = merged_df[['ct', 'disp', 'elec', 'pol', 'pauli']]

# Ensure that the indices of result_df match with cmm_filtered
result_df = result_df.reset_index(drop=True)
cmm_filtered = cmm_filtered.reset_index(drop=True)

result_sum = cmm_filtered.add(result_df, fill_value=0)

print(result_sum)


mae_scores = {}
r2_scores = {}

for label in ['ct', 'disp', 'elec', 'pol', 'pauli']:
    y_true = eda_filtered[label]
    y_pred = cmm_filtered[label]
    
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    mae_scores[label] = mae
    r2_scores[label] = r2

print("MAE Scores EDA VS CMM:")
print(mae_scores)

print("\nR² Scores EDA VS CMM:")
print(r2_scores)

            ct       disp        elec        pol       pauli
0   -46.910492 -53.067308 -246.866553 -21.170128  196.785538
1   -57.333864 -52.661236 -255.128966 -28.151957  222.749544
2   -57.726443 -52.430491 -253.374740 -25.929349  221.511412
3   -47.219770 -56.837378 -254.792901 -16.089018  200.447183
4   -42.109166 -54.779578 -252.027897 -13.399204  185.467136
..         ...        ...         ...        ...         ...
110 -50.022151 -47.702830 -229.051335 -25.300272  194.884287
111 -44.419600 -46.477934 -230.976503 -11.138758  176.761688
112 -43.088338 -47.109052 -230.829980 -15.215350  177.399026
113 -42.570026 -48.265133 -228.189318 -12.512768  174.728135
114 -42.396228 -49.368835 -235.232673 -10.460429  179.007839

[115 rows x 5 columns]
MAE Scores EDA VS CMM:
{'ct': 7.095601147140168, 'disp': 2.7675026054825045, 'elec': 1.788711213689217, 'pol': 4.066392884170406, 'pauli': 8.641378137189216}

R² Scores EDA VS CMM:
{'ct': 0.8162871138739514, 'disp': 0.9692613932024818, 'elec': 

In [4]:
#DFT VS CMM+delta
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score


cmm_filtered = pd.read_csv('cmm_filtered.csv')
eda_filtered = pd.read_csv('eda_filtered.csv')
result_df = pd.read_csv('result_df.csv')

cmm_filtered = cmm_filtered.reset_index(drop=True)
result_df = result_df.reset_index(drop=True)


result_sum = cmm_filtered.add(result_df, fill_value=0)


mae_scores = {}
r2_scores = {}


for label in ['ct', 'disp', 'elec', 'pol', 'pauli']:
    y_true = eda_filtered[label]
    y_pred = result_sum[label]
    

    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
   
    mae_scores[label] = mae
    r2_scores[label] = r2


print("MAE Scores:", mae_scores)
print("R² Scores:", r2_scores)

MAE Scores: {'ct': 2.9812737991511766, 'disp': 6.6611998054825055, 'elec': 4.539753808422505, 'pol': 21.391475205909536, 'pauli': 9.207459523680345}
R² Scores: {'ct': 0.9640774499391718, 'disp': 0.8300254357969716, 'elec': 0.9945376598675756, 'pol': -1.5891998590920302, 'pauli': 0.9813100716980013}


In [5]:
#2b terms of DFT VS CMM

import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score


df = pd.read_csv('ion_water_mbe_eda_wb97xv_qzvppd.csv')
br_df = df[df['ion_labels'] == 'F']
eda = br_df[['ct_iw', 'disp_iw', 'elec_iw', 'pol_iw', 'pauli_iw', 'ct_ww', 'disp_ww', 'elec_ww', 'pol_ww', 'pauli_ww', 'index']]
terms = ['ct', 'disp', 'elec', 'pol', 'pauli']


summed_df = pd.DataFrame()


for term in terms:

    matching_columns = [col for col in eda.columns if col.startswith(term)]
    summed_df[term] = eda[matching_columns].sum(axis=1)


summed_df['index'] = eda['index']
summed_df.to_csv('summed_df_2b.csv', index=False)
print(summed_df)


df_eda = pd.read_csv('ion_water_mbe_eda_cmm.csv')

if 'index' not in df_eda.columns:
    df_eda['index'] = range(1, len(df_eda) + 1) 


cmm = df_eda[df_eda['index'].isin(eda['index'])]
cmm_2b = cmm[['ct_2b', 'disp', 'elec_2b', 'pol_2b', 'pauli', 'index']]

print(cmm_2b)

mae_scores = {}
r2_scores = {}


for term in ['ct', 'disp', 'elec', 'pol', 'pauli']:
    y_true = summed_df[term] 
    if term in ['ct', 'elec', 'pol']:
        y_pred = cmm_2b[f'{term}_2b']
    else:
        y_pred = cmm_2b[term]  
    
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    mae_scores[term] = mae
    r2_scores[term] = r2

print("MAE Scores:", mae_scores)
print("R² Scores:", r2_scores)


            ct       disp        elec        pol       pauli  index
35  -51.108963 -50.272490 -243.063217 -54.501793  214.306692   36.0
36  -51.108963 -50.272490 -243.063217 -54.501793  214.306692   37.0
37  -57.358843 -49.482600 -251.542734 -55.147419  232.227127   38.0
38  -50.754517 -52.343188 -246.907696 -56.295817  216.501219   39.0
39  -48.358007 -51.359034 -243.156525 -56.401386  206.330784   40.0
..         ...        ...         ...        ...         ...    ...
986 -50.622849 -44.704207 -227.792161 -51.264149  205.208819  987.0
987 -48.568332 -43.659297 -226.348638 -52.674498  193.370937  988.0
988 -47.267615 -44.328967 -224.057290 -51.566491  194.381047  989.0
989 -46.759656 -45.297562 -224.527629 -51.166993  193.017758  990.0
990 -46.999068 -45.020913 -227.705449 -52.377032  194.690774  991.0

[115 rows x 6 columns]
         ct_2b       disp     elec_2b     pol_2b       pauli  index
35  -52.643424 -49.308560 -240.946370 -58.686736  219.948464     36
36  -57.513679 -48.43914

In [6]:
#2b terms of DFT VS CMM+delta
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score

df = pd.read_csv('ion_water_mbe_eda_wb97xv_qzvppd.csv')
br_df = df[df['ion_labels'] == 'F']
eda = br_df[['ct_iw', 'disp_iw', 'elec_iw', 'pol_iw', 'pauli_iw', 'ct_ww', 'disp_ww', 'elec_ww', 'pol_ww', 'pauli_ww', 'index']]

terms = ['ct', 'disp', 'elec', 'pol', 'pauli']

summed_df = pd.DataFrame()

for term in terms:
    matching_columns = [col for col in eda.columns if col.startswith(term)]
    summed_df[term] = eda[matching_columns].sum(axis=1)

summed_df['index'] = eda['index']
df_eda = pd.read_csv('ion_water_mbe_eda_cmm.csv')
if 'index' not in df_eda.columns:
    df_eda['index'] = range(1, len(df_eda) + 1)
cmm = df_eda[df_eda['index'].isin(eda['index'])]
cmm_2b = cmm[['ct_2b', 'disp', 'elec_2b', 'pol_2b', 'pauli', 'index']]
cmm_2b = cmm_2b.rename(columns={'ct_2b': 'ct', 'elec_2b': 'elec', 'pol_2b': 'pol'})

print("CMM 2-body DataFrame (cmm_2b) after renaming:")
print(cmm_2b)
result_df = pd.read_csv('result_df.csv') #predicted delta
cmm_2b = cmm_2b.reset_index(drop=True)
result_df = result_df.reset_index(drop=True)


result_sum = cmm_2b.add(result_df, fill_value=0)


mae_scores = {}
r2_scores = {}


for label in ['ct', 'disp', 'elec', 'pol', 'pauli']:
    y_true = summed_df[label]
    y_pred = result_sum[label]
    

    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
   
    mae_scores[label] = mae
    r2_scores[label] = r2


print("MAE Scores:", mae_scores)
print("R² Scores:", r2_scores)

CMM 2-body DataFrame (cmm_2b) after renaming:
            ct       disp        elec        pol       pauli  index
35  -52.643424 -49.308560 -240.946370 -58.686736  219.948464     36
36  -57.513679 -48.439142 -249.439437 -58.949904  234.871471     37
37  -57.879137 -48.604490 -250.610498 -58.916077  235.622684     38
38  -52.078961 -50.920825 -243.795929 -60.681780  221.427226     39
39  -50.193031 -50.301583 -239.480760 -60.892771  212.698973     40
..         ...        ...         ...        ...         ...    ...
986 -51.124872 -43.814097 -226.709757 -54.670041  208.373299    987
987 -49.796666 -42.762941 -224.661607 -56.757523  198.708965    988
988 -48.671122 -43.440101 -221.438207 -55.905593  199.708251    989
989 -48.153340 -44.366602 -222.779227 -55.647157  199.028894    990
990 -48.514740 -43.623144 -225.034374 -56.622236  199.645944    991

[115 rows x 6 columns]
MAE Scores: {'ct': 3.280267078233899, 'disp': 2.9782914514590737, 'elec': 4.33522208311477, 'pol': 13.519550875639